In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np


In [3]:
# 1. Hyper Parameters
input_size = 28
sequence_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10

learning_rate = 0.01
batch_size = 100
ephoc_size = 2


In [4]:
# 2. Preparing datasets
    # MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

    # Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [13]:
# 3. Build the model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fnn = nn.Linear(hidden_size, num_classes)

    def forward(self, x):

        out, _ = self.lstm(x) #out = 100x28x128
        #print(out)
        #print(out[:, -1, :])  # 100x128 take the last hidden layer
        output = self.fnn(out[:, 1, :]) #output = 100x10
        #print(output)
        
        return output

In [14]:
# 4. Generate the model
model = RNNModel(input_size, hidden_size, num_layers, num_classes)

In [15]:
# 5. Set Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
# 6. Train
for ephoc in range(ephoc_size):
    for idx, (images, labels) in enumerate(train_loader):
        # convert dataset as for Pytorch type
        images = Variable(images.view(-1, sequence_size, input_size))
        labels = Variable(labels)
        
        # Forward, Backward and Gradient decent
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        if idx%100 == 0:
            print("loss:", loss.item())

loss: 2.3080055713653564
loss: 2.3074851036071777
loss: 2.2860116958618164
loss: 2.306962013244629
loss: 2.3031535148620605
loss: 2.2594046592712402
loss: 2.306853771209717
loss: 2.303359031677246
loss: 2.2794272899627686
loss: 2.276759147644043
loss: 2.304727077484131
loss: 2.3074300289154053


In [10]:
# 7. Test
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_size, input_size))
    
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += len(predicted)
    correct += (predicted == labels).sum()
    
print("accuracy:", correct.item()/total)

accuracy: 0.9753
